## Actuator Control Allocation Example

When controlling physical systems like spacecraft or automobiles, the controller commands a desired control input $u \in \mathbb{R}^m$, which might be a thrust vector, for example. 

Usually, multiple atuators are available to produce this control input. The vector $x \in \mathbb{R}^n$ contains the single actuator input values in its components. If $n > m$ and the actuators are in general position, we call the system "over-actuated", since there are many realizations of $x$ that result in the same value of $u$ via the linear mapping $A$. 

Having this freedom of choice, we want to minimize energy consumption, modeled as $\kappa^\intercal | x |$ (with $\kappa \geq 0$), while discouraging rapid changes of the actuation values, i.e., $\Vert x-x^\mathrm{prev} \Vert_2^2$ with $x^\mathrm{prev}$ being the actuation of the previous time step. Given the bounds $x^\mathrm{min/max}$ on $x$, there might be cases when the desired control input $u$ is infeasible. Hence, we only softly penalize deviations between desired and actual control input with the cost term $\Vert A x - u \Vert_2^2$. The resulting optimization problem is the following:

\begin{align}
\min_{x}. \quad &\Vert A x - u \Vert_2^2  + \lambda^\mathrm{sm} \Vert x-x^\mathrm{prev} \Vert_2^2 + \kappa^\intercal | x |\\
\text{s.t.} \quad &x^\mathrm{min} \leq x \leq x^\mathrm{max}
\end{align}

We perform an epigraph reformulation to obtain a quadratic objective function:

\begin{align}
\min_{x, t}. \quad &\Vert A x - u \Vert_2^2  + \lambda^\mathrm{sm} \Vert x-x^\mathrm{prev} \Vert_2^2 + \kappa^\intercal t\\
\text{s.t.} \quad &x^\mathrm{min} \leq x \leq x^\mathrm{max} \\
&| x | \leq t
\end{align}

Let's define the corresponding CVXPY problem:

In [4]:
import cvxpy as cp

# define dimensions
n, m = 3, 4

# define variables
x = cp.Variable(n, name='x')
t = cp.Variable(n, name='t')

# define parameters
A = cp.Parameter((m, n), name='A')
u = cp.Parameter(m, name='u')
lamb_sm = cp.Parameter(nonneg=True, name='lamb_sm')
kappa = cp.Parameter(n, name='kappa')
x_prev = cp.Parameter(n, name='x_prev')
x_min = cp.Parameter(n, name='x_min')
x_max = cp.Parameter(n, name='x_max')

# define objective
objective = cp.Minimize(cp.sum_squares(A@x-u) + lamb_sm*cp.sum_squares(x-x_prev) + kappa@t)

# define constraints
constraints = [x_min <= x, x <= x_max, cp.abs(x) <= t]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
# TODO

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='actuator_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from actuator_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('actuator_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
# TODO

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)